In [36]:
import pandas as pd
import numpy as np
import math

In [37]:
text = ['In recent years, researchers, for computer vision, have proposed many deep learning (DL) methods for various tasks, and facial recognition (FR) made an enormous leap using these techniques.', 
        'Deep FR systems benefit from the hierarchical architecture of the deep learning methods to learn discriminative face representation.', 
        'Computer vision methods have been widely used for facial recognition',
        'Deep learning based computer vision methods have been used for facial recognition']

In [38]:
label = ['DL', 'DL', 'CV', 'CV']

In [39]:
df = pd.DataFrame({'text' : text, 'label' : label})

# Cleaning the dataset
df['text'] = df['text'].str.replace('\W', ' ') # Removes punctuation
df['text'] = df['text'].str.lower()            # Convert to Lowercase
df['text'] = df['text'].str.split()            # Split into words

In [40]:
df_train = df.iloc[0:3]
df_test = df.iloc[3:4]

print('Size of training set =', len(df_train))
print('Size of testing set =', len(df_test))

Size of training set = 3
Size of testing set = 1


In [41]:
df_train

,text,label
0,"[in, recent, years, researchers, for, computer...",DL
1,"[deep, fr, systems, benefit, from, the, hierar...",DL
2,"[computer, vision, methods, have, been, widely...",CV


In [42]:
vocabulary = set()
for sms in df_train['text']:
  for word in sms:
    vocabulary.add(word)

In [43]:
# N_Vocabulary
n_vocabulary = len(vocabulary)
print(f'Length of vocabulary = {n_vocabulary}')

Length of vocabulary = 42


In [44]:
word_counts_per_sms = {unique_word: [0] * len(df_train['text']) for unique_word in vocabulary}

for index, sms in enumerate(df_train['text']):
  for word in sms:
      word_counts_per_sms[word][index] += 1  

word_counts = pd.DataFrame(word_counts_per_sms)
df_train = pd.concat([df_train, word_counts], axis=1)

# Isolating DL and CV messages first
DL_messages = df_train[df_train['label'] == 'DL']
CV_messages = df_train[df_train['label'] == 'CV']

# P(DL) and P(CV)
p_DL = len(DL_messages) / len(df_train)
p_CV = len(CV_messages) / len(df_train)

# N_DL
n_words_per_DL_message = DL_messages['text'].apply(len)
n_DL = n_words_per_DL_message.sum()

# N_CV
n_words_per_CV_message = CV_messages['text'].apply(len)
n_CV = n_words_per_CV_message.sum()

# Laplace smoothing
alpha = 1

In [54]:
fp = open("Vocabulary.txt", "w")
fp.writelines(["%s\n" % item  for item in vocabulary])
fp.close()

In [110]:
def multinomial():

  # Initiate parameters
  parameters_DL = {unique_word:0 for unique_word in vocabulary}
  parameters_CV = {unique_word:0 for unique_word in vocabulary}

  # Calculate parameters
  for word in vocabulary:

    n_word_given_DL = DL_messages[word].sum() # spam_messages already defined
    p_word_given_DL = (n_word_given_DL + alpha) / (n_DL + alpha*n_vocabulary)
    parameters_DL[word] = p_word_given_DL

    n_word_given_CV = CV_messages[word].sum() # ham_messages already defined
    p_word_given_CV = (n_word_given_CV + alpha) / (n_CV + alpha*n_vocabulary)
    parameters_CV[word] = p_word_given_CV  
  
  smooth_DL = 1/(n_DL + alpha*n_vocabulary)
  smooth_CV = 1/(n_CV + alpha*n_vocabulary)

  p_DL_given_message = math.log(p_DL)
  p_CV_given_message = math.log(p_CV)
  message = df.iloc[3, 0]
  label = df.iloc[3, 1]

  for word in message:
    
    if word in parameters_DL:
      p_DL_given_message += math.log(parameters_DL[word])
    if word in parameters_CV:
      p_CV_given_message += math.log(parameters_CV[word])

  print(f'Probability of CV: {np.exp(p_CV_given_message}'))
  print(f'Probability of DL: {np.exp(p_DL_given_message}'))

  if p_CV_given_message >= p_DL_given_message:
      predicted = 'CV'
  else:
      predicted = 'DL'

  return predicted, parameters_DL, parameters_CV

In [111]:
def multivariate():
  
  # Initiate parameters
  parameters_DL = {unique_word:0 for unique_word in vocabulary}
  parameters_CV = {unique_word:0 for unique_word in vocabulary}

  # Calculate parameters
  for word in vocabulary:
    n_word_given_DL = DL_messages[word].sum() # spam_messages already defined
    p_word_given_DL = (n_word_given_DL + 1) / (n_DL + 2)
    parameters_DL[word] = p_word_given_DL

    n_word_given_CV = CV_messages[word].sum() # ham_messages already defined
    p_word_given_CV = (n_word_given_CV + 1) / (n_CV + 2)
    parameters_CV[word] = p_word_given_CV  
  
  smooth_DL = 1/(n_DL + 2)
  smooth_CV = 1/(n_CV + 2)

  p_DL_given_message = math.log(p_DL)
  p_CV_given_message = math.log(p_CV)
  message = df.iloc[3, 0]
  label = df.iloc[3, 1]

  for word in message:
    if word in parameters_DL:
      p_DL_given_message += math.log(parameters_DL[word])
    else:
      p_DL_given_message += math.log(1 - smooth_DL)
    if word in parameters_CV:
      p_CV_given_message += math.log(parameters_CV[word])
    else:
      p_CV_given_message += math.log(1 - smooth_CV)

  print(f'Probability of CV: {np.exp(p_CV_given_message}'))
  print(f'Probability of DL: {np.exp(p_DL_given_message}'))

  if p_CV_given_message >= p_DL_given_message:
      predicted = 'CV'
  else:
      predicted = 'DL'
    
  return predicted, parameters_DL, parameters_CV

In [122]:
predicted, parameters_DL, parameters_CV = multinomial()
print(f'Predicted Class : {predicted}')
print(parameters_DL)
print(parameters_CV)

2.2704499163162622e-17
9.40097896853843e-19
Predicted Class : CV
{'years': 0.022727272727272728, 'from': 0.022727272727272728, 'vision': 0.022727272727272728, 'proposed': 0.022727272727272728, 'dl': 0.022727272727272728, 'tasks': 0.022727272727272728, 'techniques': 0.022727272727272728, 'various': 0.022727272727272728, 'learn': 0.022727272727272728, 'facial': 0.022727272727272728, 'representation': 0.022727272727272728, 'deep': 0.045454545454545456, 'for': 0.03409090909090909, 'these': 0.022727272727272728, 'in': 0.022727272727272728, 'been': 0.011363636363636364, 'made': 0.022727272727272728, 'to': 0.022727272727272728, 'discriminative': 0.022727272727272728, 'methods': 0.03409090909090909, 'have': 0.022727272727272728, 'systems': 0.022727272727272728, 'architecture': 0.022727272727272728, 'of': 0.022727272727272728, 'face': 0.022727272727272728, 'and': 0.022727272727272728, 'many': 0.022727272727272728, 'researchers': 0.022727272727272728, 'an': 0.022727272727272728, 'hierarchical': 

In [123]:
with open("Multinomial.txt", 'w') as f: 
  f.write('Predicted Class: %s\n' % (predicted))
  f.write('\n\nConditional Probability DL Class\n')
  for key, value in parameters_DL.items(): 
    f.write('%s : %s\n' % (key, value))
  f.write('\n\nConditional Probability CV Class\n')
  for key, value in parameters_CV.items(): 
    f.write('%s : %s\n' % (key, value))

In [124]:
predicted, parameters_DL, parameters_CV = multivariate()
print(f'Predicted Class : {predicted}')
print(parameters_DL)
print(parameters_CV)

2.1055542658231758e-10
7.239115138838499e-16
Predicted Class : CV
{'years': 0.041666666666666664, 'from': 0.041666666666666664, 'vision': 0.041666666666666664, 'proposed': 0.041666666666666664, 'dl': 0.041666666666666664, 'tasks': 0.041666666666666664, 'techniques': 0.041666666666666664, 'various': 0.041666666666666664, 'learn': 0.041666666666666664, 'facial': 0.041666666666666664, 'representation': 0.041666666666666664, 'deep': 0.08333333333333333, 'for': 0.0625, 'these': 0.041666666666666664, 'in': 0.041666666666666664, 'been': 0.020833333333333332, 'made': 0.041666666666666664, 'to': 0.041666666666666664, 'discriminative': 0.041666666666666664, 'methods': 0.0625, 'have': 0.041666666666666664, 'systems': 0.041666666666666664, 'architecture': 0.041666666666666664, 'of': 0.041666666666666664, 'face': 0.041666666666666664, 'and': 0.041666666666666664, 'many': 0.041666666666666664, 'researchers': 0.041666666666666664, 'an': 0.041666666666666664, 'hierarchical': 0.041666666666666664, 'rec

In [125]:
with open("Multivariate.txt", 'w') as f: 
  f.write('Predicted Class: %s\n' % (predicted))
  f.write('\n\nConditional Probability DL Class\n')
  for key, value in parameters_DL.items(): 
    f.write('%s : %s\n' % (key, value))
  f.write('\n\nConditional Probability CV Class\n')
  for key, value in parameters_CV.items(): 
    f.write('%s : %s\n' % (key, value))